In [21]:
import pandas as pd
from datetime import datetime, timedelta, time
import numpy as np
from scipy.ndimage.interpolation import shift
from collections import defaultdict
import seaborn as sns
import sys
import time
from bs4 import BeautifulSoup
import re
%matplotlib inline

In [5]:
url = "http://web.mta.info/developers/turnstile.html"

response = requests.get(url)
dom = BeautifulSoup(response.content, 'lxml')

urls_list = []
urls = dom.find_all("div")[11].find_all("a")
for i, url in enumerate(urls):
    url_detail = str(urls[i]).split('"')[1]
    urls_list.append(url_detail)

In [6]:
urls_list[:5]

['data/nyct/turnstile/turnstile_201107.txt',
 'data/nyct/turnstile/turnstile_201031.txt',
 'data/nyct/turnstile/turnstile_201024.txt',
 'data/nyct/turnstile/turnstile_201017.txt',
 'data/nyct/turnstile/turnstile_201010.txt']

## Web crawling for the past 5 years (2015~2020)

**Field Description**

- C/A = Control Area (A002)
- UNIT = Remote Unit for a station (R051)
- SCP = Subunit Channel Position represents an specific address for a device (02-00-00)
- DATEn = Represents the date (MM-DD-YY)
- TIMEn = Represents the time (hh:mm:ss) for a scheduled audit event
- DEScn = Represent the "REGULAR" scheduled audit event (occurs every 4 hours)
- ENTRIESn = The comulative entry register value for a device
- EXISTn = The cumulative exit register value for a device

In [60]:
pd.set_option('display.expand_frame_repr', False)

In [103]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_201107.txt')

In [105]:
#get rid of white spaces in column names
data.columns = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION','DATE', 'TIME', 'DESC', 'ENTRIES','EXITS']
df = data.rename(columns= lambda x : x.strip().lower())

df['date_time'] = df['date'] + ' ' + df['time']
df['date_time'] = [datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in df['date_time']]

In [106]:
df['year'] = [x.year for x in df['date_time']]
df['weekday'] = df[['date_time']].apply(lambda x: datetime.strftime(x['date_time'], '%A'), axis=1)

### map station to boroughs

In [31]:
stations_df = pd.read_csv("https://raw.githubusercontent.com/toddwschneider/nyc-subway-turnstile-data/16d45bab6104b443bffb2f62bedad4ad587d3e96/lib/stations.csv")
stations_df.head(5)

,station,line_names,division,borough
0,1 AV,L,BMT,Manhattan
1,1 AVE,L,BMT,Manhattan
2,103 ST,1,IRT,Manhattan
3,103 ST,6,IRT,Manhattan
4,103 ST,BC,IND,Manhattan


In [32]:
len(stations_df["station"].unique()), len(data["STATION"].unique())

(572, 572)

In [109]:
mapping = dict(stations_df[['station', 'borough']].values)
df['borough'] = df.station.map(mapping)
df.head(5)

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time,year,weekday,borough
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,00:00:00,REGULAR,7481098,2545767,2020-10-31 00:00:00,2020,Saturday,Brooklyn
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,04:00:00,REGULAR,7481103,2545767,2020-10-31 04:00:00,2020,Saturday,Brooklyn
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,08:00:00,REGULAR,7481117,2545786,2020-10-31 08:00:00,2020,Saturday,Brooklyn
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,12:00:00,REGULAR,7481158,2545831,2020-10-31 12:00:00,2020,Saturday,Brooklyn
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,16:00:00,REGULAR,7481285,2545868,2020-10-31 16:00:00,2020,Saturday,Brooklyn


In [111]:
df["borough"].unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'New Jersey',
       'Staten Island'], dtype=object)

### get daily ridership

In [112]:
# Create group ID for distinct turnstiles
df['group'] = df['c/a'].astype(str) + \
                df['unit'].astype(str) + \
                df['scp'].astype(str) + \
                df['station'].astype(str)  + \
                df['linename'].astype(str) + \
                df['division'].astype(str) + \
                df['year'].astype(str)
                
# Map 'group' string to integer id     
groups = set(df['group'])


def groups_dict(groups):
    group_dict = defaultdict(int)
    for i in enumerate(list(groups)):
        group_dict[i[1]]= i[0]

    return group_dict

group_id_dict = groups_dict(groups)

df['group_id'] = [group_id_dict[x] for x in df['group']]

In [113]:
# Create station ID for distinct stations
df['station_line'] = df['station'].astype(str) + \
                df['linename'].astype(str)

In [114]:
df.head(10)

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time,year,weekday,borough,group,group_id,station_line
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,00:00:00,REGULAR,7481098,2545767,2020-10-31 00:00:00,2020,Saturday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,04:00:00,REGULAR,7481103,2545767,2020-10-31 04:00:00,2020,Saturday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,08:00:00,REGULAR,7481117,2545786,2020-10-31 08:00:00,2020,Saturday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,12:00:00,REGULAR,7481158,2545831,2020-10-31 12:00:00,2020,Saturday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,16:00:00,REGULAR,7481285,2545868,2020-10-31 16:00:00,2020,Saturday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,20:00:00,REGULAR,7481453,2545895,2020-10-31 20:00:00,2020,Saturday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/01/2020,00:00:00,REGULAR,7481511,2545902,2020-11-01 00:00:00,2020,Sunday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/01/2020,03:00:00,REGULAR,7481511,2545904,2020-11-01 03:00:00,2020,Sunday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/01/2020,07:00:00,REGULAR,7481514,2545916,2020-11-01 07:00:00,2020,Sunday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/01/2020,11:00:00,REGULAR,7481539,2545945,2020-11-01 11:00:00,2020,Sunday,Brooklyn,A002R05102-00-0059 STNQR456WBMT2020,2476,59 STNQR456W


### get lagged and net counts for distinct turnstiles

In [115]:
# Sort values by group id and date to find diff in turnstile counts from prev row
df.sort_values(['group_id','date_time'], inplace=True)
df.reset_index(drop=True)

def find_diff_prev_row(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    col_diff = abs(col_array - col_array_shifted)

    return col_diff

df['entries_diff'] = find_diff_prev_row(df['entries'])
df['exit_diff'] = find_diff_prev_row(df['exits'])

In [116]:
# Identify first rows for each group partition to use as mask when setting invalid values to nan
def find_first_rows_groups(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    first_row_mask = col_array != col_array_shifted

    return first_row_mask


df['first_row_group'] = find_first_rows_groups(df['group_id'])

# Make entries_diff and exit_diff nan when first row in group or negative value
df.loc[df['first_row_group'], 'entries_diff'] = None
df.loc[df['entries_diff'] < 0, 'entries_diff'] = None

df.loc[df['first_row_group'], 'exit_diff'] = None
df.loc[df['exit_diff'] < 0, 'exit_diff'] = None

### Outlier handling
- I will define outlier here as values > 1.5 IQR from the 75th Q.

In [117]:
df.describe()

,entries,exits,year,group_id,entries_diff,exit_diff
count,2.131050e+05,2.131050e+05,213105.0,213105.000000,2.081310e+05,2.081310e+05
mean,4.227504e+07,3.472123e+07,2020.0,2487.053762,8.694892e+02,1.030669e+04
std,2.176603e+08,1.976627e+08,0.0,1435.174875,2.955574e+05,3.132235e+06
min,0.000000e+00,0.000000e+00,2020.0,0.000000,0.000000e+00,0.000000e+00
25%,2.516820e+05,1.026370e+05,2020.0,1246.000000,1.000000e+00,3.000000e+00
50%,1.663637e+06,9.812640e+05,2020.0,2487.000000,1.800000e+01,1.900000e+01
75%,6.301980e+06,4.184440e+06,2020.0,3727.000000,6.600000e+01,6.200000e+01
max,2.128734e+09,2.123136e+09,2020.0,4973.000000,1.338188e+08,1.056965e+09


In [118]:
def find_outliers(df_series, multiple_IQR):
    """
    For a series of numerical values, remove the zeros and identify the upper outliers 
    to return a mask for all outliers in series
    """
    non_zeros = df_series.replace(0, None)
    
    adjusted_IQR = (non_zeros.quantile(.75) - non_zeros.quantile(.25)) * multiple_IQR
    outlier_lim = non_zeros.quantile(.75) + adjusted_IQR
    print(outlier_lim)
    
    outliers = [True if x > outlier_lim else False for x in df_series]
    
    outlier_count = sum(outliers)
    all_data_count = len(df_series)
    print('{} outliers identified: {} of all data'.format(outlier_count, round(outlier_count/all_data_count,6)))
    
    return outliers

In [119]:
print('Entries Outliers')
df['entries_outlier'] = find_outliers(df['entries_diff'], 5)

print('\n Exit Outliers')
df['exit_outlier'] = find_outliers(df['exit_diff'], 5)

Entries Outliers
427.0
961 outliers identified: 0.00451 of all data

 Exit Outliers
386.0
2298 outliers identified: 0.010783 of all data


In [120]:
print('All Data Len:', len(df))

clean_df = df.loc[(~df['entries_outlier'])].copy()
print('Excluding Outliers Len:', len(clean_df))

print('Keeping', round(len(clean_df)/len(df), 6))

All Data Len: 213105
Excluding Outliers Len: 212144
Keeping 0.99549


### Find missing values

In [121]:
print('Null entry diffs', clean_df.entries_diff.isnull().sum())
print('Null exit diffs', clean_df.exit_diff.isnull().sum())
print('Clean Data len:', len(clean_df))

Null entry diffs 4974
Null exit diffs 4974
Clean Data len: 212144


In [122]:
clean_df.dropna(subset = ['entries_diff', 'exit_diff'], how='any', inplace=True)

print('Null entry diffs', clean_df.entries_diff.isnull().sum())
print('Null exit diffs', clean_df.exit_diff.isnull().sum())
print('Clean Data len:', len(clean_df))

Null entry diffs 0
Null exit diffs 0
Clean Data len: 207170


In [123]:
thrown_away = len(df) - len(clean_df)
print("We're throwing away {} data points - about {} of the total".format(thrown_away, round(thrown_away/len(df), 4)))

We're throwing away 5935 data points - about 0.0279 of the total


In [124]:
clean_df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time,year,weekday,borough,group,group_id,station_line,entries_diff,exit_diff,first_row_group,entries_outlier,exit_outlier
127115,PTH16,R550,01-01-05,LACKAWANNA,1,PTH,10/31/2020,06:16:37,REGULAR,13399,4227,2020-10-31 06:16:37,2020,Saturday,New Jersey,PTH16R55001-01-05LACKAWANNA1PTH2020,0,LACKAWANNA1,1.0,1.0,False,False,False
127116,PTH16,R550,01-01-05,LACKAWANNA,1,PTH,10/31/2020,10:28:37,REGULAR,13413,4230,2020-10-31 10:28:37,2020,Saturday,New Jersey,PTH16R55001-01-05LACKAWANNA1PTH2020,0,LACKAWANNA1,14.0,3.0,False,False,False
127117,PTH16,R550,01-01-05,LACKAWANNA,1,PTH,10/31/2020,14:40:37,REGULAR,13438,4244,2020-10-31 14:40:37,2020,Saturday,New Jersey,PTH16R55001-01-05LACKAWANNA1PTH2020,0,LACKAWANNA1,25.0,14.0,False,False,False
127118,PTH16,R550,01-01-05,LACKAWANNA,1,PTH,10/31/2020,18:52:37,REGULAR,13455,4258,2020-10-31 18:52:37,2020,Saturday,New Jersey,PTH16R55001-01-05LACKAWANNA1PTH2020,0,LACKAWANNA1,17.0,14.0,False,False,False
127119,PTH16,R550,01-01-05,LACKAWANNA,1,PTH,10/31/2020,23:04:37,REGULAR,13461,4267,2020-10-31 23:04:37,2020,Saturday,New Jersey,PTH16R55001-01-05LACKAWANNA1PTH2020,0,LACKAWANNA1,6.0,9.0,False,False,False


In [125]:
borough_daily_count = clean_df.groupby(['borough', 'date']).sum()
borough_daily_count.head()

entries         exits     year  group_id  entries_diff  exit_diff  first_row_group  entries_outlier  exit_outlier
borough date                                                                                                                              
Bronx   10/31/2020  250227224956  177866494676  4395520   5317280      103441.0    91997.0                0                0            11
        11/01/2020  324219722570  231834435533  5627720   6779172       82360.0    89037.0                0                0             0
        11/02/2020  300143002932  213417723867  5249980   6342511      177781.0   155468.0                0                0            31
        11/03/2020  298465134306  213050139999  5266140   6363168      169602.0   152594.0                0                0            22
        11/04/2020  298872965319  211876859751  5258060   6356176      181746.0   161701.0                0                0            32

In [129]:
daily_count =  clean_df.groupby(['date']).sum()
daily_count.reset_index(inplace=True)
daily_count.head(10)

,date,entries,exits,year,group_id,entries_diff,exit_diff,first_row_group,entries_outlier,exit_outlier
0,10/31/2020,1064201913300,873149193596,53275480,65563477,982253.0,1.058034e+09,0,0,134
1,11/01/2020,1393988519044,1148728839879,65419720,80447330,826688.0,9.527950e+05,0,0,36
2,11/02/2020,1255244289709,1030878226053,60327300,74419094,1599266.0,1.651659e+06,0,0,417
3,11/03/2020,1256119958335,1032792574161,59818260,73699731,1504956.0,1.565327e+06,0,0,339
4,11/04/2020,1253766543548,1027937695301,59783920,73613651,1564554.0,1.619165e+06,0,0,373
5,11/05/2020,1251454762248,1022703476339,59529400,73272477,1641914.0,1.705581e+06,0,0,448
6,11/06/2020,1260039970198,1028199027063,60329320,74337539,1663945.0,1.747967e+06,0,0,457


In [70]:
data = data.drop(columns=['C/A', 'UNIT', 'SCP', 'ENTRIES', 'DIVISION', 'LINENAME', 'EXITS', 'entry_lagged', 'exit_lagged'])
data.head(2)

,DATE_TIME,STATION,DESC,borough,net_exit_counts,net_entry_counts
0,2020-10-31 00:00:00,59 ST,REGULAR,Brooklyn,NaN,NaN
1,2020-10-31 04:00:00,59 ST,REGULAR,Brooklyn,0.0,5.0


## Combine All together

In [144]:
url = "http://web.mta.info/developers/turnstile.html"

response = requests.get(url)
dom = BeautifulSoup(response.content, 'lxml')

urls_list = []
urls = dom.find_all("div")[11].find_all("a")
for i, url in enumerate(urls):
    url_detail = str(urls[i]).split('"')[1]
    urls_list.append(url_detail)

def compile_all_data(urls_list):
    all_df = pd.DataFrame()
    base_url = "http://web.mta.info/developers/"
    
    for i in range(306):
        url = base_url + urls_list[i]
        data = pd.read_csv(url)
        all_df = all_df.append(data, ignore_index=True, sort=False)
    
    return all_df
    

df = compile_all_data(urls_list)

In [145]:
df.to_pickle('subway_raw.pkl')

In [2]:
df = pd.read_pickle('subway_raw.pkl')

### aggregate functions to be used

In [22]:
def groups_dict(groups):
    group_dict = defaultdict(int)
    for i in enumerate(list(groups)):
        group_dict[i[1]]= i[0]

    return group_dict

def find_diff_prev_row(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    col_diff = abs(col_array - col_array_shifted)

    return col_diff

def find_first_rows_groups(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    first_row_mask = col_array != col_array_shifted

    return first_row_mask

def find_outliers(df_series, multiple_IQR):
    """
    For a series of numerical values, remove the zeros and identify the upper outliers 
    to return a mask for all outliers in series
    """
    non_zeros = df_series.replace(0, None)
    
    adjusted_IQR = (non_zeros.quantile(.75) - non_zeros.quantile(.25)) * multiple_IQR
    outlier_lim = non_zeros.quantile(.75) + adjusted_IQR
    print(outlier_lim)
    
    outliers = [True if x > outlier_lim else False for x in df_series]
    
    outlier_count = sum(outliers)
    all_data_count = len(df_series)
    print('{} outliers identified: {} of all data'.format(outlier_count, round(outlier_count/all_data_count,6)))
    
    return outliers

In [3]:
len(df)

60895677

In [10]:
df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/07/2020,03:00:00,REGULAR,7484683,2547321,11/07/2020 03:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/07/2020,07:00:00,REGULAR,7484689,2547330,11/07/2020 07:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/07/2020,11:00:00,REGULAR,7484721,2547382,11/07/2020 11:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/07/2020,15:00:00,REGULAR,7484818,2547419,11/07/2020 15:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/07/2020,19:00:00,REGULAR,7484998,2547445,11/07/2020 19:00:00


In [13]:
#get rid of white spaces in column names
df.columns = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION','DATE', 'TIME', 'DESC', 'ENTRIES','EXITS']
df = df.rename(columns= lambda x : x.strip().lower())

df['date_time'] = df['date'] + ' ' + df['time']
df['date_time'] = [datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in df['date_time']]

df['year'] = [x.year for x in df['date_time']]
df['weekday'] = df[['date_time']].apply(lambda x: datetime.strftime(x['date_time'], '%A'), axis=1)

In [14]:
len(df)

60895677

In [15]:
# just in case something goes wrong....
data = df.copy()

In [23]:
# Create group ID for distinct turnstiles
df['group'] = df['c/a'].astype(str) + \
                df['unit'].astype(str) + \
                df['scp'].astype(str) + \
                df['station'].astype(str)  + \
                df['linename'].astype(str) + \
                df['division'].astype(str) + \
                df['year'].astype(str)
                
# Map 'group' string to integer id     
groups = set(df['group'])
group_id_dict = groups_dict(groups)

df['group_id'] = [group_id_dict[x] for x in df['group']]
# Create station ID for distinct stations
df['station_line'] = df['station'].astype(str) + df['linename'].astype(str)

In [24]:
# Sort values by group id and date to find diff in turnstile counts from prev row
df.sort_values(['group_id','date_time'], inplace=True)
df.reset_index(drop=True)

df['entries_diff'] = find_diff_prev_row(df['entries'])
df['exit_diff'] = find_diff_prev_row(df['exits'])

# Identify first rows for each group partition to use as mask when setting invalid values to nan
df['first_row_group'] = find_first_rows_groups(df['group_id'])

# Make entries_diff and exit_diff nan when first row in group or negative value
df.loc[df['first_row_group'], 'entries_diff'] = None
df.loc[df['entries_diff'] < 0, 'entries_diff'] = None

df.loc[df['first_row_group'], 'exit_diff'] = None
df.loc[df['exit_diff'] < 0, 'exit_diff'] = None

In [25]:
print('Entries Outliers')
df['entries_outlier'] = find_outliers(df['entries_diff'], 5)

print('\n Exit Outliers')
df['exit_outlier'] = find_outliers(df['exit_diff'], 5)

print('All Data Len:', len(df))

clean_df = df.loc[(~df['entries_outlier'])].copy()
print('Excluding Outliers Len:', len(clean_df))

print('Keeping', round(len(clean_df)/len(df), 6))

Entries Outliers
1311.0
411168 outliers identified: 0.006752 of all data

 Exit Outliers
876.0
1018255 outliers identified: 0.016721 of all data
All Data Len: 60895677
Excluding Outliers Len: 60484509
Keeping 0.993248


In [26]:
print('Null entry diffs', clean_df.entries_diff.isnull().sum())
print('Null exit diffs', clean_df.exit_diff.isnull().sum())
print('Clean Data len:', len(clean_df))

Null entry diffs 31785
Null exit diffs 31785
Clean Data len: 60484509


In [27]:
clean_df.dropna(subset = ['entries_diff', 'exit_diff'], how='any', inplace=True)

print('Null entry diffs', clean_df.entries_diff.isnull().sum())
print('Null exit diffs', clean_df.exit_diff.isnull().sum())
print('Clean Data len:', len(clean_df))

thrown_away = len(df) - len(clean_df)
print("We're throwing away {} data points - about {} of the total".format(thrown_away, round(thrown_away/len(df), 4)))

Null entry diffs 0
Null exit diffs 0
Clean Data len: 60452724
We're throwing away 442953 data points - about 0.0073 of the total


In [28]:
clean_df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,...,date_time,year,group,group_id,station_line,entries_diff,exit_diff,first_row_group,entries_outlier,exit_outlier
40769704,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,01/01/2017,04:00:00,REGULAR,2598121,...,2017-01-01 04:00:00,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,6.0,28.0,False,False,False
40769705,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,01/01/2017,08:00:00,REGULAR,2598135,...,2017-01-01 08:00:00,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,14.0,7.0,False,False,False
40769706,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,01/01/2017,12:00:00,REGULAR,2598150,...,2017-01-01 12:00:00,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,15.0,6.0,False,False,False
40769707,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,01/01/2017,16:00:00,REGULAR,2598190,...,2017-01-01 16:00:00,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,40.0,25.0,False,False,False
40769708,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,01/01/2017,20:00:00,REGULAR,2598222,...,2017-01-01 20:00:00,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,32.0,33.0,False,False,False


In [29]:
clean_df['date'] = pd.to_datetime(clean_df['date'])

In [33]:
mapping = dict(stations_df[['station', 'borough']].values)
clean_df['borough'] = clean_df.station.map(mapping)

In [ ]:
# clean_df = clean_df[~(clean_df['date'] < '01/01/2015')]
# clean_df = clean_df[~(clean_df['date'] < '01/01/2015')]

In [34]:
clean_df.to_pickle('subway_clean.pkl')

In [35]:
borough_daily_count = clean_df.groupby(['borough', 'date']).sum()
borough_daily_count.head()

entries        exits     year  group_id  entries_diff  \
borough date                                                                    
Bronx   2015-01-03  46549805459  33477568721  4221425  33628139      229183.0   
        2015-01-04  57286135015  41321605309  5057650  40270902      206817.0   
        2015-01-05  60992030098  42419103650  5190640  41376933      469136.0   
        2015-01-06  61399644072  48462380324  5144295  40999848      448554.0   
        2015-01-07  57291982555  41331758634  5003245  39739903      448274.0   

                    exit_diff  first_row_group  entries_outlier  exit_outlier  
borough date                                                                   
Bronx   2015-01-03   134947.0                0                0             0  
        2015-01-04   140493.0                0                0             0  
        2015-01-05   273662.0                0                0            13  
        2015-01-06   268940.0                0                0            13  
        2015-01-07   265300.0                0                0            10

In [36]:
borough_daily_count.loc["Bronx"]

,entries,exits,year,group_id,entries_diff,exit_diff,first_row_group,entries_outlier,exit_outlier
date,,,,,,,,,
2015-01-03,46549805459,33477568721,4221425,33628139,229183.0,134947.0,0,0,0
2015-01-04,57286135015,41321605309,5057650,40270902,206817.0,140493.0,0,0,0
2015-01-05,60992030098,42419103650,5190640,41376933,469136.0,273662.0,0,0,13
2015-01-06,61399644072,48462380324,5144295,40999848,448554.0,268940.0,0,0,13
2015-01-07,57291982555,41331758634,5003245,39739903,448274.0,265300.0,0,0,10
...,...,...,...,...,...,...,...,...,...
2020-11-09,295981218290,208629256667,5249980,39281483,186941.0,160907.0,0,0,0
2020-11-10,300088264893,213321541599,5227760,38901804,193538.0,168744.0,0,0,1
2020-11-11,298160063381,211960491027,5264120,39318193,165973.0,147072.0,0,0,0


In [38]:
bronx_df = borough_daily_count.loc["Bronx"]
manhattan_df = borough_daily_count.loc["Manhattan"]
queens_df = borough_daily_count.loc["Queens"]
brooklyn_df = borough_daily_count.loc["Brooklyn"]
staten_island_df = borough_daily_count.loc["Staten Island"]
new_jersey_df = borough_daily_count.loc["New Jersey"]

bronx_df.to_pickle('bronx_subway_count.pkl')
manhattan_df.to_pickle('manhattan_subway_count.pkl')
queens_df.to_pickle('queens_subway_count.pkl')
brooklyn_df.to_pickle('brooklyn_subway_count.pkl')
staten_island_df.to_pickle('staten_island_subway_count.pkl')
new_jersey_df.to_pickle('new_jersey_subway_count.pkl')


In [41]:
clean_df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,...,year,group,group_id,station_line,entries_diff,exit_diff,first_row_group,entries_outlier,exit_outlier,borough
40769704,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,2017-01-01,04:00:00,REGULAR,2598121,...,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,6.0,28.0,False,False,False,Bronx
40769705,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,2017-01-01,08:00:00,REGULAR,2598135,...,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,14.0,7.0,False,False,False,Bronx
40769706,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,2017-01-01,12:00:00,REGULAR,2598150,...,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,15.0,6.0,False,False,False,Bronx
40769707,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,2017-01-01,16:00:00,REGULAR,2598190,...,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,40.0,25.0,False,False,False,Bronx
40769708,R421,R427,00-00-01,MIDDLETOWN RD,6,IRT,2017-01-01,20:00:00,REGULAR,2598222,...,2017,R421R42700-00-01MIDDLETOWN RD6IRT2017,0,MIDDLETOWN RD6,32.0,33.0,False,False,False,Bronx


In [42]:
final_df = clean_df[['date', 'entries_diff', 'exit_diff', 'borough']]
final_df.head()

,date,entries_diff,exit_diff,borough
40769704,2017-01-01,6.0,28.0,Bronx
40769705,2017-01-01,14.0,7.0,Bronx
40769706,2017-01-01,15.0,6.0,Bronx
40769707,2017-01-01,40.0,25.0,Bronx
40769708,2017-01-01,32.0,33.0,Bronx


In [43]:
daily_count =  final_df.groupby(['date']).sum()
# daily_count.reset_index(inplace=True)
daily_count.head(10)

,entries_diff,exit_diff
date,,
2015-01-03,2682846.0,2175723.0
2015-01-04,2481666.0,2028416.0
2015-01-05,5104111.0,4207769.0
2015-01-06,5061648.0,4224954.0
2015-01-07,5116883.0,4251015.0
2015-01-08,5017271.0,4168100.0
2015-01-09,5170899.0,4242570.0
2015-01-10,3129723.0,4927805.0
2015-01-11,2482224.0,4398049.0


In [46]:
daily_count.shape

(2142, 2)

In [47]:
daily_count.to_pickle('Data/Subway_Data/daily_count.pkl')

### previous approach

In [ ]:
# # map station to borough
# mapping = dict(stations_df[['station', 'borough']].values)
# data['borough'] = data.STATION.map(mapping)
# data.columns = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME','DESC', 'ENTRIES','EXITS', 'BOROUGH']

# # get the lagged column to calculate net entry counts and net exit counts
# data['entries_lagged'] = data.groupby(['STATION'])['ENTRIES'].shift(1)
# data['exits_lagged'] = data.groupby(['STATION'])['EXITS'].shift(1)
# data['net_exit_counts'] = abs(data["exits_lagged"] - data["EXITS"])
# data['net_entry_counts'] = abs(data["entries_lagged"] - data["ENTRIES"])

# data.drop(columns=['ENTRIES', 'EXITS', 'entries_lagged', 'exits_lagged'], inplace=True)

# data_daily = data.groupby(by=['DATE']).sum()
# data_by_borough = data.groupby(by=['BOROUGH','DATE']).sum()
# data_daily.reset_index(inplace=True)
# data_by_borough.reset_index(inplace=True, level=['DATE'])

# #change date column to datetime to later sort by date
# data_daily['DATE'] = pd.to_datetime(data_daily['DATE'])
# data_by_borough['DATE'] = pd.to_datetime(data_by_borough['DATE'])

# #delete dates before 01/01/2015
# data_daily = data_daily[~(data_daily['DATE'] < '2015-01-01')]
# data_by_borough = data_by_borough[~(data_by_borough['DATE'] < '2015-01-01')]

# #update dataframes
# daily_df = daily_df.append(data_daily, ignore_index=True, sort=False)

# bronx_df = bronx_df.append(data_by_borough.loc["Bronx"], ignore_index=True, sort=False)
# manhattan_df = manhattan_df.append(data_by_borough.loc["Manhattan"], ignore_index=True, sort=False)
# queens_df = queens_df.append(data_by_borough.loc["Queens"], ignore_index=True, sort=False)
# brooklyn_df = brooklyn_df.append(data_by_borough.loc["Brooklyn"], ignore_index=True, sort=False)
# staten_island_df = staten_island_df.append(data_by_borough.loc["Staten Island"], ignore_index=True, sort=False)
# new_jersey_df = new_jersey_df.append(data_by_borough.loc["New Jersey"], ignore_index=True, sort=False)

# # sort by date
# daily_df.sort_values(by=['DATE'], inplace=True)
# bronx_df.sort_values(by=['DATE'], inplace=True)
# manhattan_df.sort_values(by=['DATE'], inplace=True)
# queens_df.sort_values(by=['DATE'], inplace=True)
# brooklyn_df.sort_values(by=['DATE'], inplace=True)
# staten_island_df.sort_values(by=['DATE'], inplace=True)
# new_jersey_df.sort_values(by=['DATE'], inplace=True)

In [106]:
# # export csv
# daily_df.reset_index(drop=True, inplace=True)
# bronx_df.reset_index(drop=True, inplace=True)
# manhattan_df.reset_index(drop=True, inplace=True)
# queens_df.reset_index(drop=True, inplace=True)
# brooklyn_df.reset_index(drop=True, inplace=True)
# staten_island_df.reset_index(drop=True, inplace=True)
# new_jersey_df.reset_index(drop=True, inplace=True)

# daily_df.to_csv('daily_subway_count', index=False)
# bronx_df.to_csv('bronx_subway_count', index=False)
# manhattan_df.to_csv('manhattan_subway_count', index=False)
# queens_df.to_csv('queens_subway_count', index=False)
# brooklyn_df.to_csv('brooklyn_subway_count', index=False)
# staten_island_df.to_csv('staten_island_subway_count', index=False)
# new_jersey_df.to_csv('new_jersey_subway_count', index=False)